<a href="https://colab.research.google.com/github/hhauschild/keras-playground/blob/master/liveproject_dl_predict_2_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 create data frames


In [1]:
# create dataframes
import pandas as pd

teamurl = r"https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Teams.csv"
dfteams = pd.read_csv(teamurl, header=None, names=["ncaa_division","team"]);
print(dfteams.head())

gameurl = r"https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Games.csv"
dateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d")
#dfgames = pd.read_csv(gameurl, header = None, names=["date","home_team","home_score", "away_team","away_score" ], parse_dates=["date"], date_parser=dateparser, escapechar= "&");
dfgames = pd.read_csv(gameurl, header = None, names=["date","home_team","home_score", "away_team","away_score" ], parse_dates=["date"], dtype={"home_team":str,"home_score":int,"away_team":str,"away_score":int});
dfgames.head()

  ncaa_division         team
0  America East      Vermont
1  America East  Stony Brook
2  America East         UMBC
3  America East     Hartford
4  America East       Albany


,date,home_team,home_score,away_team,away_score
0,2015-11-13,Hawaii,87,Montana St.,76
1,2015-11-13,Eastern Mich.,70,Vermont,50
2,2015-11-13,Columbia,107,Kean,62
3,2015-11-13,La.-Monroe,88,McMurry,43
4,2015-11-13,Yale,70,Fairfield,57


2. Clean data: inspect team names

In [2]:
print(sorted(dfteams.team.unique()))
print(sorted(dfgames.home_team.unique()))
#print(sorted(dfgames.away_team.unique()))

['Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama State', 'Albany', 'Alcorn State', 'American University', 'Appalachian State', 'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State', 'Arkansas-Little Rock', 'Arkansas-Pine Bluff', 'Army', 'Auburn', 'Austin Peay', 'BYU', 'Ball State', 'Baylor', 'Belmont', 'Bethune-Cookman', 'Binghamton', 'Boise State', 'Boston College', 'Boston University', 'Bowling Green', 'Bradley', 'Brown', 'Bryant University', 'Bucknell', 'Buffalo', 'Butler', 'Cal Poly', 'Cal State Bakersfield', 'Cal State Fullerton', 'Cal State Northridge', 'California', 'California Baptist', 'Campbell', 'Canisius', 'Central Arkansas', 'Central Connecticut State', 'Central Michigan', 'Charleston', 'Charleston Southern', 'Charlotte', 'Chattanooga', 'Chicago State', 'Cincinnati', 'Citadel', 'Clemson', 'Cleveland State', 'Coastal Carolina', 'Colgate', 'Colorado', 'Colorado State', 'Columbia', 'Connecticut', 'Coppin State', 'Cornell', 'Creighton', 'Dartmo

2.1 clean HTML escaping


In [3]:
import html
html.unescape('A&amp;M-Corpus Chris')

'A&M-Corpus Chris'

In [4]:

dfgames.home_team=dfgames.home_team.apply(lambda x:html.unescape(x))
dfgames.away_team=dfgames.away_team.apply(lambda x:html.unescape(str(x)))

print(sorted(dfgames.home_team.unique()))
print(sorted(dfgames.away_team.unique()))

['A&M-Corpus Chris', 'Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama St.', 'Alabama State', 'Alas. Anchorage', 'Albany', 'Albany (NY)', 'Alcorn St.', 'Alcorn State', 'American', 'Appalachian St.', 'Arizona', 'Arizona St.', 'Ark.-Pine Bluff', 'Arkansas', 'Arkansas St.', 'Arkansas State', 'Army', 'Army West Point', 'Auburn', 'Austin Peay', 'Ave Maria', 'BYU', 'Bakersfield', 'Ball St.', 'Ball State', 'Baylor', 'Belmont', 'Bethune-Cookman', 'Binghamton', 'Boise St.', 'Boise State', 'Boston College', 'Boston U.', 'Bowling Green', 'Bradley', 'Brown', 'Bryant', 'Bucknell', 'Buffalo', 'Butler', 'CSU Bakersfield', 'CSU Fullerton', 'CSUN', 'Cal Baptist', 'Cal Poly', 'Cal St. Fullerton', 'Cal St. Northridge', 'California', 'Campbell', 'Canisius', 'Cent. Conn. St.', 'Centenary (LA)', 'Central Ark.', 'Central Conn. St.', 'Central Mich.', 'Chaminade', 'Charleston', 'Charleston So.', 'Charlotte', 'Chattanooga', 'Chicago St.', 'Chicago State', 'Cincinnati', 'Citadel', 'Cl

2.2 text normalzation

In [20]:
normalizedTeamCache={}



we normalize the team names by looking up their Wikipedia pages using DuckDuckGo, and take the name of the page as new team name.

In [8]:
from bs4 import BeautifulSoup
import urllib
import re
import time

def wpname(s):
  if s in normalizedTeamCache:
    return normalizedTeamCache[s]
  #print("lookup "+s)
  q=urllib.parse.quote("wikipedia "+s+" men's basketball").replace("%20","+")
  #todo: cache results!
  time.sleep(1) # do not flood duckduckgo
  site = urllib.request.urlopen('http://duckduckgo.com/html?q='+q+'&kl=us-en&t=ffab&df=y')
  data = site.read()
  parsed = BeautifulSoup(data)
  first_link = parsed.findAll('div', {'class': re.compile('links_main*')})[0].a['href']
  linkRe = re.compile(r'.*uddg=')
  link = urllib.parse.unquote( re.sub(".*uddg=","",first_link))
  wikipediaName = re.sub(".*wiki/","",link)
  #print("found "+wikipediaName)
  normalizedTeamCache[s]=wikipediaName
  return wikipediaName

In [15]:
import re
import string
punctuation_table = str.maketrans('','',string.punctuation)
print("updating dfteams")
dfteams.team=dfteams.team.apply(lambda x:wpname(str(x).strip().translate(punctuation_table)))
print("updating dfgames.home_team")
dfgames.home_team=dfgames.home_team.apply(lambda x:wpname(str(x).strip().translate(punctuation_table)))
print("updating dfgames.away_team")
dfgames.away_team=dfgames.away_team.apply(lambda x:wpname(str(x).strip().translate(punctuation_table)))

print(sorted(dfteams.team.unique()))
print(sorted(dfgames.home_team.unique()))
print(sorted(dfgames.away_team.unique()))


updating dfteams
updating dfgames.home_team
updating dfgames.away_team
['1926%E2%80%9327_Vanderbilt_Commodores_men%27s_basketball_team', '1972_Olympic_Men%27s_Basketball_Final', '1973_ACC_Men%27s_Basketball_Tournament', '2018%E2%80%9319_UMKC_Kangaroos_men%27s_basketball_team', '2019%E2%80%9320_Siena_Saints_men%27s_basketball_team', '2019_SEC_Men%27s_Basketball_Tournament', 'Abilene_Christian_Wildcats_men%27s_basketball', 'Alabama_Crimson_Tide_men%27s_basketball', 'Alabama_State_Hornets_basketball', 'Albany_Great_Danes_men%27s_basketball', 'Alcorn_State_Braves_basketball', 'American_Eagles_men%27s_basketball', 'Appalachian_State_Mountaineers_men%27s_basketball', 'Arizona_State_Sun_Devils_men%27s_basketball', 'Arizona_Wildcats_men%27s_basketball', 'Arkansas%E2%80%93Pine_Bluff_Golden_Lions_men%27s_basketball', 'Arkansas_Razorbacks_men%27s_basketball', 'Arkansas_State_Red_Wolves_men%27s_basketball', 'Army_Black_Knights_men%27s_basketball', 'Auburn_Tigers_men%27s_basketball', 'Austin_Peay_G

3 add calculated columns


In [16]:
dfteams["home_points"] = dfteams.team.apply(lambda row: dfgames.query("home_team=='"+row+"'").home_score.sum())
dfteams["away_points"] = dfteams.team.apply(lambda row: dfgames.query("away_team=='"+row+"'").away_score.sum())
#dfteams["home_point_diff"] = dfteams.team.apply(lambda row: dfgames.query("home_team=='"+row+"'").apply(lambda x:x["home_score"]-x["away_score"]).sum())
dfteams

,ncaa_division,team,home_points,away_points
0,America East,Vermont_Catamounts_men%27s_basketball,4991,5103
1,America East,Stony_Brook_Seawolves_men%27s_basketball,4560,4400
2,America East,UMBC_Retrievers_men%27s_basketball,4971,4792
3,America East,Hartford_Hawks_men%27s_basketball,4377,4765
4,America East,Albany_Great_Danes_men%27s_basketball,4732,4852
...,...,...,...,...
348,Western Athletic,California_Baptist_Lancers_men%27s_basketball,930,1184
349,Western Athletic,Cal_State_Bakersfield_Roadrunners_men%27s_bask...,3572,3174
350,Western Athletic,Seattle_Redhawks_men%27s_basketball,5640,3330
351,Western Athletic,2018%E2%80%9319_UMKC_Kangaroos_men%27s_basketb...,4719,4521


4 write csv

In [19]:
dfteams.to_csv('Games-Calculated.csv')